# Test

In [64]:
import pandas as pd
import re

In [65]:
all_format = ["Basic","Sequential", "Encoded", "Wide"]

In [66]:
class TransactionDf():

    def __init__(self,file_path,header=False,target_column=None,separator=",",formatting="Basic"):
        self.file_paths = [file_path]
        self.headers = [header]
        self.target_columns = [target_column]
        self.separators = [separator]
        self.dfs = []
        self.load_transactions(file_path,header=header,target_column=target_column,sep=separator,formatting=formatting)
    
    def load_transaction_csv(self,file_path,header,target_column,sep,formatting):
        if header:
            transactions = pd.read_csv(file_path)
        else :
            transactions = pd.read_csv(file_path,header=None)
        
        if formatting == "Wide":
            #   transaction_id, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
            #   items, "bread,milk,eggs", "bread,butter", ...
            formatting = "Basic"
            transactions = transactions.T

        if formatting == "Basic":
            #   transaction_id,items
            #   1,"bread,milk,eggs"
            #   2,"bread,butter"

            # check if there is a transaction_id column
            if len(transactions.columns) == 1:
                target_column = transactions.columns[0]
            elif target_column not in transactions.columns:
                raise ValueError(f"the targeted column {target_column} is not in {transactions.columns}")
            if sep:
                transactions = transactions[transactions[target_column].apply(lambda x: isinstance(x, str))]
                transactions[target_column] = transactions[target_column].apply(lambda x: [i.strip() for i in x.split(",")])
            

            # all this hot encode a column and keep all the others column
            transactions_exploded = transactions[target_column].explode()

            transactions_dummies = pd.get_dummies(transactions_exploded)

            #0_ in order to make a sequential feature
            transactions_dummies.columns = ['0_' + col for col in transactions_dummies.columns]

            transactions_dummies = transactions_dummies.groupby(transactions_dummies.index).sum()

            transactions = transactions.drop(columns=[target_column]).join(transactions_dummies)
        elif formatting == "Sequential":
            #   client_id, sequence
            #    C1,"(bread,milk),(butter)"
            #    C2,"(bread),(butter,cheese)"
            all_items = set()
            parsed_sequences = []

            for seq in transactions[target_column]:
                all_transactions = re.findall(r"\((.*?)\)", seq)
                steps = [t.split(",") for t in all_transactions]
                steps = [[item.strip() for item in step if item.strip()] for step in steps]
                parsed_sequences.append(steps)
                for step in steps:
                    all_items.update(step)

            all_items = sorted(all_items)
            print(all_items)
            print("seq", parsed_sequences)

            output_rows = []

            for i in range(len(transactions)):
                client_id = transactions.loc[i, "client_id"]
                steps = parsed_sequences[i]

                row = {"client_id": client_id}
                max_steps = len(steps)
                for i in range(max_steps):
                    for item in all_items:
                        row[f"{i}_{item}"] = 1 if item in steps[i] else 0
                output_rows.append(row)
            transactions = pd.DataFrame(output_rows).fillna(0)
        
        # format encoded need nothing
        return transactions

    def load_transactions(self,file_path,header=True,target_column=None,sep=",",formatting="Basic"):
        " Give pandas dataset of transaction "
        res = None
        if file_path[-4:] == ".csv":
            try :
                res = self.load_transaction_csv(file_path,header,target_column,sep,formatting)
            except ValueError as e:
                print(f"Error: {e}")
        
        
        if res is not None:
            self.dfs.append(res)
    
    def displays(self):
        for df in self.dfs:
            display(df)
    
    def combine(self,indexes=[],column_to_check=None):
        "Combine 2 Databases given index [x,y] with a column to keep (transaction name), display database"
        if len(self.dfs) == 2:
            indexes = [0,1]
        elif (indexes == [] or max(indexes) >= len(self.dfs)) :
            return
        res = None
        if column_to_check is not None:
            # Merge using outer join to keep all names
            df_merged = pd.merge(self.dfs[indexes[0]], self.dfs[indexes[1]], on=column_to_check, how='outer', suffixes=('_1', '_2'))

            df_merged = df_merged.fillna(0)
            numeric_cols = [c for c in df_merged.columns if c != column_to_check]
            df_merged[numeric_cols] = df_merged[numeric_cols].astype(int)

            # Combine columns with similar meaning
            all_columns = set(self.dfs[indexes[0]].columns).union(self.dfs[indexes[1]].columns) - {column_to_check}
            for col in all_columns:
                # Find all columns corresponding to this original column (with _1/_2 suffixes or exact match)
                cols_to_sum = [c for c in df_merged.columns if c == col or c.startswith(col + "_")]
                if cols_to_sum:
                    df_merged[col] = df_merged[cols_to_sum].sum(axis=1)
                    if len(cols_to_sum) == 2: 
                        df_merged.drop(columns=cols_to_sum, inplace=True, errors='ignore')
            res = df_merged
            res = res[sorted(res.columns)]

        return res

        

In [69]:
T_df = TransactionDf('../data/stupid.csv',header=True,target_column="Articles")
T_df.displays()

,Transaction,0_beurre,0_confiture,0_lait,0_pain
0,T1,0,0,1,1
1,T2,1,0,0,1
2,T3,1,1,1,0
3,T4,1,0,1,1
4,T5,1,0,1,0


In [70]:
T_df.load_transactions('../data/wide.csv',header=True,target_column="sequence",formatting="Wide")
T_df.displays()
#display(T_df.combine(column_to_check="Transaction"))

,Transaction,0_beurre,0_confiture,0_lait,0_pain
0,T1,0,0,1,1
1,T2,1,0,0,1
2,T3,1,1,1,0
3,T4,1,0,1,1
4,T5,1,0,1,0


,0_bread,0_butter,0_cheese,0_eggs,0_items,0_milk,0_yogurt
transaction_id,0,0,0,0,1,0,0
1,1,0,0,1,0,1,0
2,1,1,0,0,0,0,0
3,0,1,1,0,0,1,0
4,1,1,0,0,0,1,0
5,1,0,0,1,0,0,0
6,0,0,1,0,0,1,1
7,1,1,0,1,0,1,0
8,0,0,1,0,0,0,1
9,1,1,0,1,0,0,0
